In [1]:
import os
import pandas as pd
from scipy import stats
from datetime import datetime
import numpy as np
import traceback

# pd.set_option('display.max_columns', None) 
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

Load LifeLines and REDCap variables

In [2]:
#Lifelines
path_high=os.getcwd()+"/high_BMI_lifelines.csv"
path_low=os.getcwd()+"/low_BMI_lifelines.csv"
path_noage=os.getcwd()+"/LifeLines_BMI_export_latest.csv"

#Redcap
path_redcap=os.getcwd()+"/BMI_6-6-2023.csv"
path_gender=os.getcwd()+"/BMI_all_participants_with_filled_scan_tab.csv"

Get LifeLines dfs

In [3]:
high_BMI=pd.read_csv(path_high)
# high_BMI

In [4]:
low_BMI=pd.read_csv(path_low)
low_BMI=low_BMI[~low_BMI['patientID'].isin([130781,334327])] #Exclude cases with 10 nods in first reading
# low_BMI

In [5]:
BMI_lifelines=pd.concat([high_BMI,low_BMI])

New version (age missing)

In [6]:
# lifelines_noage=pd.read_csv(path_noage)
# lifelines_noage=lifelines_noage[~lifelines_noage['patientID'].isin([130781,334327])] #Exclude cases with 10 nods in first reading

# columns_to_check = ['bodylength_cm_all_m_1','bodyweight_kg_all_m_1']

# # Filtering
# baseline_values = lifelines_noage[lifelines_noage[columns_to_check].notna().any(axis=1)]
# baseline_values

In [7]:
# # Convert 'date' column to datetime format
# baseline_values['date']=pd.to_datetime(baseline_values['date'])

# # Find the index of the earliest date within each group
# earliest_index = baseline_values.groupby('patientID')['date'].idxmin()

# # Select the rows with the earliest date for each group
# earliest_dates = baseline_values.loc[earliest_index]
# earliest_dates

In [8]:
# earliest_dates.count()

Load REDCap variables

In [9]:
BMI_red=pd.read_csv(path_redcap)
BMI_red=BMI_red[BMI_red['participant_id'].isin(list(high_BMI['patientID'].values)+ list(low_BMI['patientID'].values))]

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3106: DtypeWarning: Columns (17,18,39,48,58,90,91,127,164,201,238,241,242,275,349) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
BMI_gender=pd.read_csv(path_gender)
BMI_gender=BMI_gender[BMI_gender['participant_id'].isin(list(high_BMI['patientID'].values)+ list(low_BMI['patientID'].values))]

assert np.sum(BMI_gender['participant_gender___1']==1)+np.sum(BMI_gender['participant_gender___2']==1)==len(BMI_gender) #1 male, 2 female

In [11]:
BMI_gender['participant_gender___1']=BMI_gender['participant_gender___1'].map({1:'Male', 0:'Female'})
BMI_gender.rename(columns={'participant_gender___1': 'gender'}, inplace=True)
# BMI_gender

In [12]:
BMI_gender_high=BMI_gender[BMI_gender['participant_id'].isin(list(high_BMI['patientID'].values))]
BMI_gender_low=BMI_gender[BMI_gender['participant_id'].isin(list(low_BMI['patientID'].values))]

Calculate age for LifeLines and REDCap and compare them

In [13]:
# #Function to calculate age of participant from age in which he/she was born
def calculate_age(scan_date, birth_date):
    try:
        birth_date=datetime(int(birth_date[:4]),int(birth_date[5:7]),int(birth_date[8:10]),0,0)
        scan_date=datetime(int(scan_date[:4]),int(scan_date[5:7]),int(scan_date[8:10]),0,0)
        return int(scan_date.year - birth_date.year - ((scan_date.month, scan_date.day) < (birth_date.month, birth_date.day)))
    except:
        return np.nan

In [14]:
BMI_red['age_at_scan']=BMI_red.apply(lambda row: calculate_age(row['scan_date'],row['date_of_birth']), axis=1)

In [15]:
BMI_red_high=BMI_red[BMI_red['participant_id'].isin(high_BMI['patientID'])]
BMI_red_low=BMI_red[BMI_red['participant_id'].isin(low_BMI['patientID'])]

In [16]:
#High BMI
total_null=0
for index,age in enumerate(high_BMI['age_at_scan'].values):
    if age==BMI_red_high['age_at_scan'].iloc[index]:
        pass
    else:
        if age<200: #Ensure no nan values are printed
            print('From RedCap scan date is',BMI_red_high['scan_date'].iloc[index], 'and date of birth is',BMI_red_high['date_of_birth'].iloc[index])
            print("RedCap age is:",calculate_age(BMI_red_high['scan_date'].iloc[index],BMI_red_high['date_of_birth'].iloc[index]))
            print('From LifeLines age at scan is',age)
            print('Participant is:',high_BMI['patientID'].iloc[index])
            print('\n')
        else:
            total_null+=1
        #     # print('From RedCap scan date is',BMI_red_high['scan_date'].iloc[index], 'and date of birth is',BMI_red_high['date_of_birth'].iloc[index])
        #     # print('From LifeLines age at scan is',age)
        #     print('Nan values in LifeLines data for participant',high_BMI['patientID'].iloc[index])
        #     print('\n')

print('Total null values in LifeLines data:',total_null)
assert high_BMI.isnull()['age_at_scan'].sum()==total_null

From RedCap scan date is 2020-04-30 and date of birth is 1967-06-22
RedCap age is: 52
From LifeLines age at scan is 50.0
Participant is: 103302


From RedCap scan date is 2020-06-07 and date of birth is 1967-06-23
RedCap age is: 52
From LifeLines age at scan is 50.0
Participant is: 108772


From RedCap scan date is 2020-06-07 and date of birth is 1969-12-19
RedCap age is: 50
From LifeLines age at scan is 47.0
Participant is: 112498


Total null values in LifeLines data: 22


In [17]:
#Low BMI
total_null=0
for index,age in enumerate(low_BMI['age_at_scan'].values):
    
    if age==BMI_red_low['age_at_scan'].iloc[index]:
        pass
    else:
        if age<200: #Ensure no nan values are printed
            print('From RedCap scan date is',BMI_red_low['scan_date'].iloc[index], 'and date of birth is',BMI_red_low['date_of_birth'].iloc[index])
            print("RedCap age is:",calculate_age(BMI_red_low['scan_date'].iloc[index],BMI_red_low['date_of_birth'].iloc[index]))
            print('From LifeLines age at scan is',age)
            print('Participant is:',low_BMI['patientID'].iloc[index])
            print('\n')
        else:
            total_null+=1
        #     # print('From RedCap scan date is',BMI_red_high['scan_date'].iloc[index], 'and date of birth is',BMI_red_high['date_of_birth'].iloc[index])
        #     # print('From LifeLines age at scan is',age)
            # print('Nan values in LifeLines data for participant',low_BMI['patientID'].iloc[index])
        #     print('\n')

print('Total null values in LifeLines data:',total_null)
assert low_BMI.isnull()['age_at_scan'].sum()==total_null

From RedCap scan date is 2020-04-23 and date of birth is 1967-03-23
RedCap age is: 53
From LifeLines age at scan is 50.0
Participant is: 103906


From RedCap scan date is 2020-05-07 and date of birth is 1962-07-21
RedCap age is: 57
From LifeLines age at scan is 55.0
Participant is: 105600


Total null values in LifeLines data: 9


Get rest of variables from REDCap - Copied from 'demographics_emphysema_statistics.ipynb'

In [18]:
# BMI_red

In [19]:
#Check if there are any missing values
age=BMI_red['age_at_scan']
weight=BMI_red['participant_weight']
height=BMI_red['participant_length']

gender=BMI_gender['gender']
never_smoker=BMI_lifelines['never_smoker']
ever_smoker=BMI_lifelines['ever_smoker']
current_smoker=BMI_lifelines['current_smoker']
ex_smoker=BMI_lifelines['ex_smoker']
pack_years=BMI_lifelines['pack_years']

try: #Confirm that there are no missing values for age, weight, height
    assert np.sum(age.value_counts().values)==len(BMI_red)
    assert np.sum(weight.value_counts().values)==len(BMI_red)
    assert np.sum(height.value_counts().values)==len(BMI_red)
except:
    print(traceback.format_exc())
    print('\n')

#Missing values for gender
try:
    assert np.sum(gender.value_counts().values)==len(BMI_lifelines)
except:
    print("For gender we have {} values out of {}".format(np.sum(gender.value_counts().values),len(BMI_lifelines)))

#Report missing values for smoking attributes
try:
    assert np.sum(never_smoker.value_counts().values)==len(BMI_lifelines)
except:
    print("For never_smoker we have {} values out of {}".format(np.sum(never_smoker.value_counts().values),len(BMI_lifelines)))

try:
    assert np.sum(ever_smoker.value_counts().values)==len(BMI_lifelines)
except:
    print("For ever_smoker we have {} values out of {}".format(np.sum(ever_smoker.value_counts().values),len(BMI_lifelines)))
    
try:
    assert np.sum(current_smoker.value_counts().values)==len(BMI_lifelines)
except:
    print("For current_smoker we have {} values out of {}".format(np.sum(current_smoker.value_counts().values),len(BMI_lifelines)))
    
try:
    assert np.sum(ex_smoker.value_counts().values)==len(BMI_lifelines)
except:
    print("For ex_smoker we have {} values out of {}".format(np.sum(ex_smoker.value_counts().values),len(BMI_lifelines)))
    
try:
    assert np.sum(pack_years.value_counts().values)==len(BMI_lifelines)
except:
    print("For pack_years we have {} values out of {}".format(np.sum(pack_years.value_counts().values),len(BMI_lifelines)))

For never_smoker we have 327 values out of 358
For ever_smoker we have 327 values out of 358
For current_smoker we have 327 values out of 358
For ex_smoker we have 327 values out of 358
For pack_years we have 177 values out of 358


In [20]:
#Create df with demographics to be filled below for each of the emphysema/non-emphysema categories
df_statistics=pd.DataFrame(columns=['All participants (N=358)','Participants with low BMI (N=178)','Participants with high BMI (N=180)','P value'],
                            index=['Sex (male, %)',#'Number of participants','Gender Male', 'Gender Female',
                                   'Age', 'Weight', 'Height',
                                   'Never smoker','Ever smoker','Pack years']) #'Current smoker', 'Ex smoker',

df_statistics.index.name = 'Characteristics' 

#All participants statistics
num_participants=len(BMI_red) #Same as BMI_lifelines
males=np.sum(BMI_gender['gender']=='Male')
females=np.sum(BMI_gender['gender']=='Female')
age_mean=np.mean(BMI_red['age_at_scan'])
age_std=np.std(BMI_red['age_at_scan'])
weight_mean=np.mean(BMI_red['participant_weight'])
weight_std=np.std(BMI_red['participant_weight'])
length_mean=np.mean(BMI_red['participant_length'])
length_std=np.std(BMI_red['participant_length'])

never_smoker=np.sum(BMI_lifelines['never_smoker']==True)
ever_smoker=np.sum(BMI_lifelines['ever_smoker']==True)
current_smoker=np.sum(BMI_lifelines['current_smoker']==True)
ex_smoker=np.sum(BMI_lifelines['ex_smoker']==True)
pack_years_mean=np.mean(BMI_lifelines['pack_years'])
pack_years_std=np.std(BMI_lifelines['pack_years'])

#High BMI participants statistics
num_participants_high=len(BMI_red_high)
males_high=np.sum(BMI_gender_high['gender']=='Male')
females_high=np.sum(BMI_gender_high['gender']=='Female')
age_mean_high=np.mean(BMI_red_high['age_at_scan'])
age_std_high=np.std(BMI_red_high['age_at_scan'])
weight_mean_high=np.mean(BMI_red_high['participant_weight'])
weight_std_high=np.std(BMI_red_high['participant_weight'])
length_mean_high=np.mean(BMI_red_high['participant_length'])
length_std_high=np.std(BMI_red_high['participant_length'])

never_smoker_high=np.sum(high_BMI['never_smoker']==True)
ever_smoker_high=np.sum(high_BMI['ever_smoker']==True)
current_smoker_high=np.sum(high_BMI['current_smoker']==True)
ex_smoker_high=np.sum(high_BMI['ex_smoker']==True)
pack_years_mean_high=np.mean(high_BMI['pack_years'])
pack_years_std_high=np.std(high_BMI['pack_years'])

#Low BMI participants statistics
num_participants_low=len(BMI_red_low)
males_low=np.sum(BMI_gender_low['gender']=='Male')
females_low=np.sum(BMI_gender_low['gender']=='Female')
age_mean_low=np.mean(BMI_red_low['age_at_scan'])
age_std_low=np.std(BMI_red_low['age_at_scan'])
weight_mean_low=np.mean(BMI_red_low['participant_weight'])
weight_std_low=np.std(BMI_red_low['participant_weight'])
length_mean_low=np.mean(BMI_red_low['participant_length'])
length_std_low=np.std(BMI_red_low['participant_length'])

never_smoker_low=np.sum(low_BMI['never_smoker']==True)
ever_smoker_low=np.sum(low_BMI['ever_smoker']==True)
current_smoker_low=np.sum(low_BMI['current_smoker']==True)
ex_smoker_low=np.sum(low_BMI['ex_smoker']==True)
pack_years_mean_low=np.mean(low_BMI['pack_years'])
pack_years_std_low=np.std(low_BMI['pack_years'])


#Add them to df
df_statistics['All participants (N=358)']=[
                     #    num_participants,
                        str(males)+' ('+str(100*np.round(males/num_participants,2))+'%)',
                     #    str(females)+' ('+str(100*np.round(females/num_participants,2))+'%)',
                        str(np.round(age_mean,1))+str('±')+str(np.round(age_std,1)),
                        str(np.round(weight_mean,1))+str('±')+str(np.round(weight_std,1)),
                        str(np.round(length_mean,1))+str('±')+str(np.round(length_std,1)),
                        str(never_smoker)+' ('+str(100*np.round(never_smoker/num_participants,2))+'%)', 
                        str(ever_smoker)+' ('+str(100*np.round(ever_smoker/num_participants,2))+'%)',
                        str(np.round(pack_years_mean,1))+str('±')+str(np.round(pack_years_std,1))]

df_statistics['Participants with high BMI (N=180)']=[
                     #    str(num_participants_high)+' ('+str(100*np.round(num_participants_high/num_participants,2))+'%)',
                        str(males_high)+' ('+str(100*np.round(males_high/num_participants_high,2))+'%)',
                     #    str(females_high)+' ('+str(100*np.round(females_high/num_participants_high,2))+'%)',
                        str(np.round(age_mean_high,1))+str('±')+str(np.round(age_std_high,1)),
                        str(np.round(weight_mean_high,1))+str('±')+str(np.round(weight_std_high,1)),
                        str(np.round(length_mean_high,1))+str('±')+str(np.round(length_std_high,1)),
                        str(never_smoker_high)+' ('+str(100*np.round(never_smoker_high/num_participants_high,1))+'%)', 
                        str(ever_smoker_high)+' ('+str(100*np.round(ever_smoker_high/num_participants_high,2))+'%)', 
                        str(np.round(pack_years_mean_high,1))+str('±')+str(np.round(pack_years_std_high,1))]

df_statistics['Participants with low BMI (N=178)']=[
                     #    str(num_participants_low)+' ('+str(100*np.round(num_participants_low/num_participants,1))+'%)',
                        str(males_low)+' ('+str(np.round(100*males_low/num_participants_low,1))+'%)',
                     #    str(females_low)+' ('+str(np.round(100*females_low/num_participants_low,1))+'%)',       
                        str(np.round(age_mean_low,1))+str('±')+str(np.round(age_std_low,1)),
                        str(np.round(weight_mean_low,1))+str('±')+str(np.round(weight_std_low,1)),
                        str(np.round(length_mean_low,1))+str('±')+str(np.round(length_std_low,1)),
                        str(never_smoker_low)+' ('+str(np.round(100*never_smoker_low/num_participants_low,1))+'%)', 
                        str(ever_smoker_low)+' ('+str(np.round(100*ever_smoker_low/num_participants_low,1))+'%)',
                        str(np.round(pack_years_mean_low,1))+str('±')+str(np.round(pack_years_std_low,1))]

df_statistics
#Symbol '±' copied from online
#'ever smoker' is 'current smoker'+'ex smoker'. 

,All participants (N=358),Participants with low BMI (N=178),Participants with high BMI (N=180),P value
Characteristics,,,,
"Sex (male, %)",67 (19.0%),24 (13.5%),43 (24.0%),NaN
Age,59.8±10.3,61.3±10.8,58.2±9.5,NaN
Weight,85.9±32.9,55.0±6.3,116.5±15.2,NaN
Height,170.9±8.6,171.2±8.3,170.6±8.8,NaN
Never smoker,138 (39.0%),82 (46.1%),56 (30.0%),NaN
Ever smoker,189 (53.0%),87 (48.9%),102 (56.99999999999999%),NaN
Pack years,15.1±14.2,15.2±13.2,15.0±14.9,NaN


In [21]:
#Manual correction of rounding errors
df_statistics['Participants with high BMI (N=180)'][-2]=df_statistics['Participants with high BMI (N=180)'][-2][:4]+'('+ str(
    np.round(float(df_statistics['Participants with high BMI (N=180)'][-2][5:10])))+'%)'


### P-value calculations

In [22]:
P_vals=['','gender=','','age=','weight=','height=','smoking status=','','pack years='] #For those column we calculate p values

In [23]:
#Create two groups of categorical data
group1 = [males_high, males_low]  # frequencies of each category in group 1 - males
group2 = [females_high, females_low]  # frequencies of each category in group 2 - females

# calculate the chi-square statistic and p-value
chi2_statistic, p_value, dof, expected = stats.chi2_contingency([group1, group2])

# print the p-value
print("P-value for gender:", p_value)
P_vals[1]=str(p_value)


# create two groups of numeric data
group1 = BMI_red_low['age_at_scan']
group2 = BMI_red_high['age_at_scan']

# calculate the t-statistic and p-value
t_statistic, p_value = stats.ttest_ind(group1, group2)

# print the p-value
print("P-value for age:", p_value)
P_vals[3]=str(p_value)


#Repeat above for other categorical and numeric data

# create two groups of numeric data
group1 = BMI_red_low['participant_weight']
group2 = BMI_red_high['participant_weight']

# calculate the t-statistic and p-value
t_statistic, p_value = stats.ttest_ind(group1, group2)

# print the p-value
print("P-value for weight:", p_value)
P_vals[4]=str(p_value)


# create two groups of numeric data
group1 = BMI_red_low['participant_length']
group2 = BMI_red_high['participant_length']

# calculate the t-statistic and p-value
t_statistic, p_value = stats.ttest_ind(group1, group2)

# print the p-value
print("P-value for length:", p_value)
P_vals[5]=str(p_value)

# create two groups of numeric data
group1 = low_BMI['pack_years'].dropna()
group2 = high_BMI['pack_years'].dropna()

# calculate the t-statistic and p-value
t_statistic, p_value = stats.ttest_ind(group1, group2)

# print the p-value
print("P-value for pack_years:", p_value)
P_vals[8]=str(p_value) #was 10


# create two groups of categorical data
group1 = [never_smoker_high, never_smoker_low]  # frequencies of each category in group 1 - never smokers
group2 = [ever_smoker_high, ever_smoker_low]  # frequencies of each category in group 2 - even smokers

# calculate the chi-square statistic and p-value
chi2_statistic, p_value, dof, expected = stats.chi2_contingency([group1, group2])

# print the p-value
print("P-value for smoking status (never, ever):", p_value)
P_vals[6]=str(p_value)

P-value for gender: 0.016920984700406223
P-value for age: 0.003412932286864418
P-value for weight: 1.1263208978785698e-162
P-value for length: 0.5338510507246161
P-value for pack_years: 0.9473732620841131
P-value for smoking status (never, ever): 0.022562494328701454


In [24]:
#P values smaller than 0.001 set to '<0.001' and others are rounded to 2 digits
for ind,val in enumerate(P_vals):
    if val!='':
        if float(val)<0.001:
            P_vals[ind]='<0.001'
        else:
            P_vals[ind]=np.round(float(val),3)
    else:
        pass

indices_to_delete = [0,2]  # Indices of the values to be deleted

# Delete values at specified indices using list comprehension and 'del'
P_vals = [value for index, value in enumerate(P_vals) if index not in indices_to_delete]

df_statistics['P value']=P_vals
df_statistics

,All participants (N=358),Participants with low BMI (N=178),Participants with high BMI (N=180),P value
Characteristics,,,,
"Sex (male, %)",67 (19.0%),24 (13.5%),43 (24.0%),0.017
Age,59.8±10.3,61.3±10.8,58.2±9.5,0.003
Weight,85.9±32.9,55.0±6.3,116.5±15.2,<0.001
Height,170.9±8.6,171.2±8.3,170.6±8.8,0.534
Never smoker,138 (39.0%),82 (46.1%),56 (30.0%),0.023
Ever smoker,189 (53.0%),87 (48.9%),102 (57.0%),
Pack years,15.1±14.2,15.2±13.2,15.0±14.9,0.947


In [25]:
df_statistics.to_excel('demographics_BMI_statistics.xlsx')